In [ ]:
import sys
# Add folder containing base gmca and hgmca files to path.
sys.path.insert(0, "../hgmca")
import numpy as np
from gmca import *
import healpy as hp
from test_helpers import cmb_test_data
import wavelet, helpers, pickle

# The frequency maps we want to write out.
freq = [23,90,350]
f_row = [0,1,6]
# Parameters for our wavelet transform
band_lim = 621
wav_b = 3
min_scale = 1
samp = 0
nside = 256
orig_cmb = hp.read_map('/n/home09/swagnercarena/cmb_maps/input_maps/maps_s2/'
	+'all_cmb_sim_id_90_GHZ.fits',verbose=False)
mask = None
mask_address = '/n/home09/swagnercarena/cmb_maps/input_maps/planck_mask.fits'
orig_cmb_mask, mask = helpers.mask_hpx_map(orig_cmb,mask_address,
	cutoff=0.5,average=False,return_mask=True,mask=mask)
# We only need one prefix since we can just clean it out each time after
# we use it.
prefix = '/n/home09/swagnercarena/cmb_maps/hgmca/f2_'
hpx_map_prefix = prefix + 'hgmca_t'
wav_map_prefix = (hpx_map_prefix + '%d%d%d%s%d%d%d%d%d%d%d'%(
		n_sources,lam_s,lam_p_v/1e10,norm[0],rseed,m_level,gmca_iterations,
		rmse_rate,use_lam_p_scaling,use_A_cmb,np.log10(A_cmb_p)))
# The initial data is constant for our other parameters, so we can
# load the data here
print('Getting data and setting up wavelet transform')
X , A_cmb, nside_list = cmb_test_data(nside_list=True,
	all_foreground_map=True,use_mask=use_mask,use_s2=True)
wav_t = wavelet.AxisymWaveletTransformation(wav_b,min_scale,
		band_lim,samp=samp)
wav_t.set_nside_list(nside_list)
# wav_t.hgmca_calc_wav_lim_list(nside,wav_map_prefix)
# print(wav_t.wav_lim_list)
wav_t.set_wav_lim_list([0, 0, 0, 2, 3, 5, 6])
lam_p = lam_p_v * np.ones(n_sources)
normalizer = helpers.NormalizationClass(norm)
X_n = normalizer.normalize_hpx_data(X,wav_t=wav_t)
A_n = normalizer.normalize_A_col(A_cmb)
rmse_error_list = []
loss_list = []
A_hier = None
S = None